<a href="https://colab.research.google.com/github/OgunSerifOnargan/arkitek_notebooks/blob/main/yolov8_multiclass_label_corrector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install roboflow
!pip install ultralytics

from roboflow import Roboflow
import os
import shutil

from IPython import display

display.clear_output()

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.1.78
    Uninstalling opencv-python-headless-4.8.1.78:
    

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 9.9 MB/s eta 0:00:00


In [ ]:
rf = Roboflow(api_key="AeZizLSORNnn25TSelEa")
project = rf.workspace("marihuanna").project("marihuanna_segmentation")
dataset = project.version(9).download("yolov8")


In [ ]:
rf = Roboflow(api_key="PfVLpb6NRwK9ocHbiLmw")
project = rf.workspace("smoking-vf33w").project("cigarette_seg_clean_v1")
dataset = project.version(16).download("yolov8")


In [ ]:
rf = Roboflow(api_key="P38FBow6u0MmnZgixAsh")
project = rf.workspace("gambling-2chss").project("gambling_version_2")
dataset = project.version(7).download("yolov8")


In [8]:
# @title
def label_changer(alone_dataset_name, prev_label, new_index):
  klasor_yolu = alone_dataset_name  # Dataset yolu
  first_index = prev_label                               # ilk indexin ne olduğu
  changed_index = new_index                              # ilk indexin ne ile değiştirileceği

  for root,dirs,files in os.walk(klasor_yolu):

      for dir in dirs:

          if dir == 'labels':
              folder_path = os.path.join(root,dir)
              dosya_listesi = os.listdir(folder_path)

              for dosya_adı in dosya_listesi:

                  if dosya_adı.endswith('.txt'):
                      dosya_yolu = os.path.join(folder_path, dosya_adı)

                      with open(dosya_yolu, 'r') as dosya:
                          satirlar = dosya.readlines()

                      with open(dosya_yolu, 'w') as dosya:

                          for satir in satirlar:

                              if satir and satir[0] == f'{first_index}':
                                  satir = f'{changed_index}' + satir[1:]
                              dosya.write(satir)

def count_files_in_directory(directory):
    total_files = 0

    for root, dirs, files in os.walk(directory):
        total_files += len(files)

    return total_files

def unite_datasets(source_roots, destination_root):
    # List of subfolders to copy
    subfolders = ["train", "valid", "test"]

    # Iterate over main datasets
    for source_root in source_roots:
        for subfolder in subfolders:
            source_subfolder = os.path.join(source_root, subfolder)
            destination_subfolder = os.path.join(destination_root, subfolder)

            # Create destination subfolder if it doesn't exist
            os.makedirs(destination_subfolder, exist_ok=True)

            subdirectories = ["images", "labels"]

            # Iterate over subdirectories
            for subdir in subdirectories:
                source_path = os.path.join(source_subfolder, subdir)
                destination_path = os.path.join(destination_subfolder, subdir)

                # Create destination subdirectory if it doesn't exist
                os.makedirs(destination_path, exist_ok=True)

                # Copy files from source to destination
                for file in os.listdir(source_path):
                    source_file = os.path.join(source_path, file)
                    destination_file = os.path.join(destination_path, file)
                    shutil.copy(source_file, destination_file)
    num_of_total_images = count_files_in_directory(destination_root) / 2

    print(f'Number of images is: {num_of_total_images}')
    print("Files have been copied to the destination folder.")



In [6]:

label_changer("/content/cigarette_seg_clean_v1-16", 0, 1)

label_changer("/content/gambling_version_2-7", 0, 2)
label_changer("/content/gambling_version_2-7", 1, 1)

In [7]:
source_roots = ["/content/marihuanna_segmentation-9",
                "/content/cigarette_seg_clean_v1-16",
                "/content/gambling_version_2-7"]

destination_root = "/content/united_dataset1"
unite_datasets(source_roots, destination_root)

Files have been copied to the destination folder.


In [9]:

# Replace '/path/to/main/folder' with the actual path to your main folder
main_folder = '/content/united_dataset1'

total_files = count_files_in_directory(main_folder)
print(f"Total files in '{main_folder}': {total_files}")


Total files in '/content/united_dataset1': 15148


In [10]:
#zaten eğitilmiş olan modeli yüklüyoruz, detaylar başka bir videonun konusu...
from ultralytics import YOLO

model = YOLO('yolov8x-seg.pt')  # load a pretrained model (recommended for training)

model.train(data="/content/united_dataset1/data.yaml",
            epochs=200,  #verisetinin üzerinden kaç defa geçildiği
            batch=15,     #kaçlı gruplar halinde alınacağını geçiyoruz (gpu ram boyutuna göre arttırılabilir)   #kaç epoch boyunca validation data performansı artmazsa duracağı #augmentation uygulama kısmı
            imgsz=1024,
            cos_lr=True,
            optimizer="AdamW",
            lr0=0.000001,
            lrf=0.0000001,
            patience=5
            )

100%|██████████| 137M/137M [00:08<00:00, 16.4MB/s]
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=segment, mode=train, model=yolov8x-seg.pt, data=/content/united_dataset1/data.yaml, epochs=200, patience=5, batch=15, imgsz=1024, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classe

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x795e3bfe2c50>
fitness: 1.2710019561361259
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'metrics/precision(M)', 'metrics/recall(M)', 'metrics/mAP50(M)', 'metrics/mAP50-95(M)']
maps: array([     1.2312,      1.0051,      1.4793])
names: {0: 'marihuanna', 1: 'smoking', 2: 'gambling'}
plot: True
results_dict: {'metrics/precision(B)': 0.8324583220978875, 'metrics/recall(B)': 0.6831035402639953, 'metrics/mAP50(B)': 0.7786256430350681, 'metrics/mAP50-95(B)': 0.6394917359134613, 'metrics/precision(M)': 0.8374196600348776, 'metrics/recall(M)': 0.6966470635537648, 'metrics/mAP50(M)': 0.7847649421993234, 'metrics/mAP50-95(M)': 0.5990225947673017, 'fitness': 1.2710019561361259}
save_dir: PosixPath('runs/segment/train')
seg: ultralytics

In [11]:

import shutil
shutil.move("/content/runs", "/content/drive/MyDrive/mari-cigar-gambling_11-10_v1")

'/content/drive/MyDrive/mari-cigar-gambling_11-10_v1'

In [ ]:
import time

# Run this cell to wait for 2 minutes
time.sleep(2 * 60)  # 2 minutes in seconds

from google.colab import runtime
runtime.unassign()


# Forcefully disconnect from the Colab runtime
def disconnect_colab():
    import os
    os.kill(os.getpid(), 9)

# Run the function to disconnect
disconnect_colab()